In [ ]:
# Importing the required lybraries and functions - Part 1

from lorenzetti_utils.read_events import *
from ROOT import TCanvas, TH1F, TH2F, TH1I, TFile
from ROOT import TLatex, gPad
from ROOT import kRed, kBlue, kBlack,TLine,kBird, kOrange,kGray, kYellow, kViolet, kGreen, kAzure
from pprint import pprint
from pandas import DataFrame

import array
import numpy as np
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

GeV=1000.

In [ ]:
# Importing the required lybraries and functions - Part 2

from Gaugi import load
from Gaugi.monet.utils import getColor,getColors
from Gaugi.monet.PlotFunctions import *
from Gaugi.monet.TAxisFunctions import *
from Gaugi.monet.AtlasStyle import *
from Gaugi.utilities import stdvector_to_list, progressbar
SetAtlasStyle()

In [ ]:
# Function to read the .root file and save the relevant variables in a DataFrame format

def read_events( path , nov=-1):
    
    event = EventStore(path, "physics")
    vars = ['et','eta','phi','reta','rphi','rhad','eratio','weta2','f1','f3']
    d = { key:[] for key in vars }
    d['rings'] = []
    d['MC_et'] = []
    d['MC_eta'] = []
    d['MC_phi'] = []
    
    if nov < 0 or nov > event.GetEntries():
        nov = event.GetEntries()
    
    for entry in progressbar( range(nov) , event.GetEntries() , prefix='Reading...') : 
        event.GetEntry(entry) 
        
        cluster_cont = event.retrieve("CaloClusterContainer_Clusters")
        truth_cont = event.retrieve("TruthParticleContainer_Particles")
        for caloRings in event.retrieve("CaloRingsContainer_Rings"):
            
            emClus = cluster_cont.at(caloRings.cluster_link)
            for key in vars:
                d[key].append( getattr(emClus,key) )    
            d['rings'].append(stdvector_to_list(caloRings.rings))
            d['MC_et'].append( getattr(truth_cont.at(caloRings.cluster_link),'et') )
            d['MC_eta'].append( getattr(truth_cont.at(caloRings.cluster_link),'eta') )
            d['MC_phi'].append( getattr(truth_cont.at(caloRings.cluster_link),'phi') )
            
    
    return DataFrame(d)

In [ ]:
# Define the path to the downloaded .root file

path = '/eos/user/e/efurtado/lorenzetti_calibration/calibration/electrons.merge.root'

e_event = read_events(path, nov=100000)
# nov = number of events read from the .root file


In [ ]:
# Importing the required lybraries and functions - Part 3

import numpy as np
from scipy import stats
from scipy.stats import wasserstein_distance
import matplotlib as mpl
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from statistics import mean, stdev

from calibtools import calc_sv, create_inputs, train_bdt, do_crossval
mpl.rcParams['figure.dpi'] = 300
